In [1]:
import pandas as pd
import scipy.stats
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [2]:
data = pd.read_excel('card transactions.xlsx', converters={'Merchnum': lambda x: str(x)})

In [3]:
data.shape

(96753, 10)

In [4]:
data.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [5]:
data.columns=['record','card','date','merch#','merchdes','merchstate','merchzip','transtype','amount','fraud']

In [6]:
data['date']=pd.to_datetime(data['date'],format='%Y-%m-%d')

In [7]:
data['card']=data['card'].astype(str)

In [8]:
data.dtypes

record                 int64
card                  object
date          datetime64[ns]
merch#                object
merchdes              object
merchstate            object
merchzip             float64
transtype             object
amount               float64
fraud                  int64
dtype: object

In [9]:
df=data.copy()

## Fix Data Type

### Remove outlier

In [10]:
df=df[df['amount']<max(df['amount'])]

In [11]:
df['amount'].sort_values(ascending=False)

47339    47900.00
59516    30372.46
80886    28392.84
89673    27218.00
1103     26910.00
           ...   
47165        0.08
57855        0.02
78387        0.01
91179        0.01
56124        0.01
Name: amount, Length: 96752, dtype: float64

### Keep only transaction type

In [12]:
df = df[df['transtype']=='P']

In [13]:
df.shape

(96397, 10)

## Filling Missing Values

In [14]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [15]:
numrecords = len(df)
df.count() * 100 /numrecords

record        100.000000
card          100.000000
date          100.000000
merch#         96.682469
merchdes      100.000000
merchstate     98.941876
merchzip       95.539280
transtype     100.000000
amount        100.000000
fraud         100.000000
dtype: float64

### Merchnum

In [16]:
merchnumnull=df[df['merch#'].isnull()]
merchnumgood=df[df['merch#'].notnull()]

In [17]:
merchnumnull.head()

,record,card,date,merch#,merchdes,merchstate,merchzip,transtype,amount,fraud
115,116,5142182128,2010-01-03,NaN,GRAINGER #973,IL,60089.0,P,327.34,0
135,136,5142126842,2010-01-03,NaN,AUTOMATED OFFICE PRODU,MD,20706.0,P,2110.00,0
168,169,5142132574,2010-01-03,NaN,ROLL CALL NEWSPAPER,DC,20001.0,P,104.69,0
201,202,5142140436,2010-01-03,NaN,MONTGOMERY COLLEGE-PHONE,MD,20850.0,P,266.00,0
257,258,5142257292,2010-01-04,NaN,ARGENT CHEMICAL LABS,WA,98052.0,P,240.00,0


In [18]:
merchnumnull.shape

(3198, 10)

In [19]:
merchnumgood.shape

(93199, 10)

In [20]:
# merchnumnull['merchdes'].unique()

In [21]:
for nulldes in merchnumnull['merchdes'].unique():
    for gooddes in merchnumgood['merchdes'].unique():
        score = fuzz.ratio(nulldes,gooddes)
        if score ==100:
            df.loc[(df['merchdes']==nulldes) &(df['merch#'].isnull()),'merch#'] =\
            list(df.loc[(df['merchdes']==nulldes) &(df['merch#'].notnull()),'merch#'])[0]
            break

In [22]:
df[df['merchdes']=='BREDE EXPOSITION SERVICES']

,record,card,date,merch#,merchdes,merchstate,merchzip,transtype,amount,fraud
12706,12707,5142257777,2010-02-22,08-0002815224,BREDE EXPOSITION SERVICES,CO,80216.0,P,389.5,0
27063,27064,5142119910,2010-04-13,08-0002815224,BREDE EXPOSITION SERVICES,CO,80216.0,P,76.0,0
82843,82844,5142150919,2010-10-22,08-0002815224,BREDE EXPOSITION SERVICES,CO,80216.0,P,113.2,0
84547,84548,5142150919,2010-11-02,08-0002815224,BREDE EXPOSITION SERVICES,CO,80216.0,P,7.7,0


In [23]:
df.count()

record        96397
card          96397
date          96397
merch#        94359
merchdes      96397
merchstate    95377
merchzip      92097
transtype     96397
amount        96397
fraud         96397
dtype: int64

In [24]:
# filling merchnum with record(same merchdes with same record num)
merchnumdf=df[df['merch#'].isnull()].groupby('merchdes').apply(lambda x: x['merch#'].fillna(list(x['record'])[0])).reset_index()
merchnumdf.head()

,merchdes,level_1,merch#
0,21ST CENTURY EXPO. GROUP,52170,52171
1,21ST CENTURY EXPO. GROUP,73141,52171
2,800.COM,23281,23282
3,A & M FIRE & SAFETY INC,14815,14816
4,A ARVIN TONER STORE,14780,14781


In [25]:
merchnumdf.count()

merchdes    2038
level_1     2038
merch#      2038
dtype: int64

In [26]:
df

,record,card,date,merch#,merchdes,merchstate,merchzip,transtype,amount,fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
...,...,...,...,...,...,...,...,...,...,...
96748,96749,5142276053,2010-12-31,3500000006160,BEST BUY 00001610,KY,41042.0,P,84.79,0
96749,96750,5142225701,2010-12-31,8090710030950,MARKUS OFFICE SUPPLIES,OH,45248.0,P,118.75,0
96750,96751,5142226486,2010-12-31,4503057341100,"TECH PAC, INC",OH,45150.0,P,363.56,0
96751,96752,5142244619,2010-12-31,8834000695412,BUY.COM,CA,92656.0,P,2202.03,0


In [27]:
df.loc[merchnumdf['level_1'],'merch#'] = merchnumdf['merch#'].values

In [28]:
df['merch#'].isnull().sum()

0

## Merchzip

In [29]:
df.merchzip

0        38118.0
1         1803.0
2        20706.0
3        38118.0
4        38118.0
          ...   
96748    41042.0
96749    45248.0
96750    45150.0
96751    92656.0
96752     7606.0
Name: merchzip, Length: 96397, dtype: float64

### Merchstate

### Filling merchstate with merchdes

In [30]:
df['merchzip']=df['merchzip'].astype(str).str[:-2]

In [31]:
# pip install uszipcode

In [32]:
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)

In [33]:
for i in df[(df['merchstate'].isnull()==True)&(df['merchzip']!='n')].index:
    zipcode=df.loc[i, 'merchzip']
    if len(zipcode)==5:
        info = search.by_zipcode(zipcode)
        if info.state is not None:
            df.loc[i, 'merchstate']=info.state

In [34]:
df.count()

record        96397
card          96397
date          96397
merch#        96397
merchdes      96397
merchstate    95391
merchzip      96397
transtype     96397
amount        96397
fraud         96397
dtype: int64

In [35]:
df.loc[df['merchzip']=='n','merchzip']=np.nan

In [36]:
statenull=df[df['merchstate'].isnull()]
stategood=df[df['merchstate'].notnull()]

In [37]:
for null in statenull['merchdes'].unique():
    for good in stategood['merchdes'].unique():
        if null == good:
            df.loc[(df['merchdes']==null) &(df['merchstate'].isnull()),'merchstate'] =\
            list(df.loc[(df['merchdes']==null) &(df['merchstate'].notnull()),'merchstate'])[0]
            break

In [38]:
df.count()

record        96397
card          96397
date          96397
merch#        96397
merchdes      96397
merchstate    96048
merchzip      92097
transtype     96397
amount        96397
fraud         96397
dtype: int64

### Filling merchstate with merchnum

In [39]:
statenull2=df[df['merchstate'].isnull()]
stategood2=df[df['merchstate'].notnull()]

In [40]:
for null in statenull2['merch#'].unique():
    for good in stategood2['merch#'].unique():
        if null == good:
            df.loc[(df['merch#']==null) &(df['merchstate'].isnull()),'merchstate'] =\
            list(df.loc[(df['merch#']==null) &(df['merchstate'].notnull()),'merchstate'])[0]
            break

In [41]:
df.count()

record        96397
card          96397
date          96397
merch#        96397
merchdes      96397
merchstate    96079
merchzip      92097
transtype     96397
amount        96397
fraud         96397
dtype: int64

### Filling merchstate with merchzip

In [42]:
statenull3=df[df['merchstate'].isnull()]
stategood3=df[df['merchstate'].notnull()]

for null in statenull3['merchzip'].unique():
    for good in stategood3['merchzip'].unique():
        if null == good:
            df.loc[(df['merchzip']==null) &(df['merchstate'].isnull()),'merchstate'] =\
            list(df.loc[(df['merchzip']==null) &(df['merchstate'].notnull()),'merchstate'])[0]
            break

In [43]:
df.count()

record        96397
card          96397
date          96397
merch#        96397
merchdes      96397
merchstate    96096
merchzip      92097
transtype     96397
amount        96397
fraud         96397
dtype: int64

### Filling merchstate with record#

In [44]:
# filling merchstate with record(same merchdes with same record num)
merchstatedf=df[df['merchstate'].isnull()].groupby('merchdes').apply(lambda x: x['merchstate'].fillna(list(x['record'])[0])).reset_index()
merchstatedf.head()

,merchdes,level_1,merchstate
0,AAFES BUCHANAN FURN/TOYS,24185,24186
1,AAFES BUCHANAN FURN/TOYS,24226,24186
2,AAFES BUCHANAN FURN/TOYS,24283,24186
3,AAFES BUCHANAN FURN/TOYS,25487,24186
4,AAFES BUCHANAN MAIN STR,24297,24298


In [45]:
df.loc[merchstatedf['level_1'],'merchstate'] = merchstatedf['merchstate'].values

In [46]:
df.count()

record        96397
card          96397
date          96397
merch#        96397
merchdes      96397
merchstate    96397
merchzip      92097
transtype     96397
amount        96397
fraud         96397
dtype: int64

## Filling merchzip with merchdes

In [358]:
zipnull=df[df['merchzip'].isnull()]
zipgood=df[df['merchzip'].notnull()]

zipnull

,record,card,date,merch#,merchdes,merchstate,merchzip,transtype,amount,fraud
51,52,5142204384,2010-01-02,5000006000095,IBM INTERNET 01000025,NY,NaN,P,20.15,0
54,55,5142146340,2010-01-02,5000006000095,IBM INTERNET 01000025,NY,NaN,P,23.90,0
55,56,5142260984,2010-01-02,5000006000095,IBM INTERNET 01000025,NY,NaN,P,19.95,0
58,59,5142204384,2010-01-02,5000006000095,IBM INTERNET 01000025,NY,NaN,P,20.15,0
59,60,5142204384,2010-01-02,5000006000095,IBM INTERNET 01000025,NY,NaN,P,20.15,0
...,...,...,...,...,...,...,...,...,...,...
96651,96652,5142151962,2010-12-30,263,RETAIL DEBIT ADJUSTMENT,GA,NaN,P,125.00,0
96668,96669,5142195081,2010-12-30,465905123335,DURHAM SAFE & LOCK CO,NC,NaN,P,103.00,0
96720,96721,5142178272,2010-12-31,23095,TRULOGIC SYSTEMS,ON,NaN,P,343.42,0
96730,96731,5142122644,2010-12-31,263,RETAIL DEBIT ADJUSTMENT,GA,NaN,P,340.79,0


In [359]:
for null in zipnull['merchdes'].unique():
    for good in zipgood['merchdes'].unique():
        if null == good:
            df.loc[(df['merchdes']==null) &(df['merchzip'].isnull()),'merchzip'] =\
            list(df.loc[(df['merchdes']==null) &(df['merchzip'].notnull()),'merchzip'])[0]
            break

In [360]:
df.count()

record        96397
card          96397
date          96397
merch#        96397
merchdes      96397
merchstate    96397
merchzip      94354
transtype     96397
amount        96397
fraud         96397
dtype: int64

### Filling merchzip with recordnum

In [361]:
merchzipdf=df[df['merchzip'].isnull()].groupby('merchdes').apply(lambda x: x['merchzip'].fillna(list(x['record'])[0])).reset_index()

In [362]:
merchzipdf

,merchdes,level_1,merchzip
0,8886538103 PREFBUYERRE,36462,36463
1,ACE TONER SPLY47950019,2014,2015
2,ACE TONER SPLY47950019,4715,2015
3,ACE TONER SPLY47950019,13932,2015
4,ACE TONER SPLY47950019,24011,2015
...,...,...,...
2038,ZYMED LABORATORIES,71231,16682
2039,ZYMED LABORATORIES,76302,16682
2040,ZYMED LABORATORIES,80845,16682
2041,ZYMED LABORATORIES,91320,16682


In [363]:
df.loc[merchzipdf['level_1'],'merchzip'] = merchzipdf['merchzip'].values

In [364]:
df.count()

record        96397
card          96397
date          96397
merch#        96397
merchdes      96397
merchstate    96397
merchzip      96397
transtype     96397
amount        96397
fraud         96397
dtype: int64

In [365]:
df.to_csv('cleaned_data.csv',index=False)